In [1]:
import nltk
import string
import pandas as pd
import numpy as np

from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_selection import mutual_info_classif, chi2, SelectKBest
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from scipy.stats import pearsonr

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

tokenizer = TreebankWordTokenizer()
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
count_vectorizer = CountVectorizer()
label_encoder = LabelEncoder()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/chamilkaudugoda/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chamilkaudugoda/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/chamilkaudugoda/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import uniform, randint
from sklearn.metrics import classification_report, accuracy_score, make_scorer, f1_score, balanced_accuracy_score
import joblib
from scipy.sparse import csr_matrix

In [5]:
df = pd.read_csv('/Users/chamilkaudugoda/Documents/Master of Business Analytics/Module 3/Text Analytics for Business/Project/labelled_newscatcher_dataset.csv', delimiter=';')
corpus = list(df['link'])
labels = list(df['topic'])
y = label_encoder.fit_transform(labels)

# Data Pre-processing

In [6]:
def preprocess_text(text):
    tokens = tokenizer.tokenize(text)
    tokens = [word.lower() for word in tokens if word not in string.punctuation]
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

corpus_norm = [preprocess_text(title) for title in corpus]

X_train, X_test, y_train, y_test = train_test_split(corpus_norm, y, train_size=0.7, random_state=42)
print(f'Unique labels: {np.unique(y_train)}')

Unique labels: [0 1 2 3 4 5 6 7]


# Feature Extraction

In [7]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

len(tfidf_vectorizer.vocabulary_)

579907

# Feauture Selection

In [8]:
selector = SelectKBest(chi2, k=1000)
X_train_reduc = selector.fit_transform(X_train_tfidf, y_train)
X_test_reduc = selector.transform(X_test_tfidf)

In [9]:
def calculate_correlations_optimized(X, threshold=0.8):
    X_dense = X.toarray()
    corr_matrix = np.corrcoef(X_dense, rowvar=False)
    to_remove = set()
    for i in range(corr_matrix.shape[0]):
        for j in range(i + 1, corr_matrix.shape[1]):
            if corr_matrix[i, j] > threshold:
                to_remove.add(j)
    return to_remove

# Identify highly correlated features
features_to_remove = calculate_correlations_optimized(X_train_reduc)

# Remove highly correlated features from TF-IDF matrices
X_train_tfidf_selected = csr_matrix(np.delete(X_train_reduc.toarray(), list(features_to_remove), axis=1))
X_test_tfidf_selected = csr_matrix(np.delete(X_test_reduc.toarray(), list(features_to_remove), axis=1))

In [10]:
print(f'Shape of X_train_tfidf_selected: {X_train_tfidf_selected.shape}')
print(f'Shape of X_test_tfidf_selected: {X_test_tfidf_selected.shape}')

Shape of X_train_tfidf_selected: (76141, 694)
Shape of X_test_tfidf_selected: (32633, 694)


# SVM with Parameter Tuning


In [11]:
# Parameter grid for SVM
svm_param_grid = {
    'C': np.logspace(-3, 3, 10),
    'kernel': ['linear', 'rbf'],
    'gamma': np.logspace(-3, 3, 10)
}

In [12]:
# Initialize SVM with One-vs-Rest approach
svm_clf = SVC(decision_function_shape='ovr')

In [13]:
# RandomizedSearchCV for SVM
svm_random_search = RandomizedSearchCV(svm_clf, param_distributions=svm_param_grid, scoring = make_scorer(balanced_accuracy_score), n_iter=20, cv=3, verbose=1, random_state=42, n_jobs=-1)
svm_random_search.fit(X_train_tfidf_selected, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


RandomizedSearchCV(cv=3, estimator=SVC(), n_iter=20, n_jobs=-1,
                   param_distributions={'C': array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]),
                                        'gamma': array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]),
                                        'kernel': ['linear', 'rbf']},
                   random_state=42,
                   scoring=make_scorer(balanced_accuracy_score), verbose=1)

In [14]:
print(svm_random_search.best_params_)

{'kernel': 'linear', 'gamma': 2.154434690031882, 'C': 215.44346900318823}


In [15]:
best_svm = svm_random_search.best_estimator_

In [16]:
cv = StratifiedKFold(n_splits=10)
cv_results = cross_val_score(best_svm, X_train_tfidf_selected, y_train, cv=cv, scoring=make_scorer(balanced_accuracy_score))

print(f'10-fold cross-validation Balanced Accuracy-score: {np.mean(cv_results)}')

10-fold cross-validation Balanced Accuracy-score: 0.6790130726912766


In [17]:
svm_predictions = best_svm.predict(X_test_tfidf_selected)

In [19]:
bal_acc=balanced_accuracy_score(y_test,svm_predictions)

In [20]:
svm_report = classification_report(y_test, svm_predictions,target_names=label_encoder.classes_)
svm_accuracy = accuracy_score(y_test, svm_predictions)
print("SVM Classification Report:\n", svm_report)
print("Balanced Accuracy: ", bal_acc)

SVM Classification Report:
                precision    recall  f1-score   support

     BUSINESS       0.74      0.61      0.67      4419
ENTERTAINMENT       0.83      0.69      0.75      4488
       HEALTH       0.63      0.73      0.67      4466
       NATION       0.39      0.73      0.51      4412
      SCIENCE       0.83      0.67      0.74      1144
       SPORTS       0.92      0.84      0.88      4551
   TECHNOLOGY       0.86      0.73      0.79      4558
        WORLD       0.70      0.43      0.53      4595

     accuracy                           0.68     32633
    macro avg       0.74      0.68      0.69     32633
 weighted avg       0.73      0.68      0.69     32633

Balanced Accuracy:  0.6778795172312739
